In [ ]:
import numpy as np

**Funciones Auxiliares**

In [ ]:
def calcularAx(A: np.array, x: np.array):
    res = np.zeros(A.shape[0])  #A.shape[0] devuelve la cantidad de filas de A
    for i, row in enumerate(A):
        for j, value in enumerate(row):
            res[i] += value * x[j][0] 
    return res

In [ ]:
def normaInf(A):
    sumatorias = []
    for i, row in enumerate(A):
        sumatorias.append(sum(abs(row)))
    
    return max(sumatorias)

**Modulo ALC**


In [ ]:
def matricesIguales(A: np.array, B: np.array, atol = 1e-8):
    if A.size != B.size and A[0].size != B[0].size:
        return False
    for i, fila in enumerate(A):
        for j, valor in enumerate(fila):
           if error(np.float64(valor), np.float64(B[i][j])) > atol:
                return False
    return True

def error(x, y):
    return abs(np.float64(x) - np.float64(y))

def error_relativo(x,y):
    if x == 0:
        return abs(y)
    return error(x,y)/abs(x)


In [ ]:
def sonIguales(x,y,atol=1e-08):
    return np.allclose(error(x,y),0,atol=atol)

In [ ]:
assert(not sonIguales(1,1.1))
assert(sonIguales(1,1 + np.finfo('float64').eps))
assert(not sonIguales(1,1 + np.finfo('float32').eps))
assert(not sonIguales(np.float16(1),np.float16(1) + np.finfo('float32').eps))
assert(sonIguales(np.float16(1),np.float16(1) + np.finfo('float16').eps,atol=1e-3))

assert(np.allclose(error_relativo(1,1.1),0.1))
assert(np.allclose(error_relativo(2,1),0.5))
assert(np.allclose(error_relativo(-1,-1),0))
assert(np.allclose(error_relativo(1,-1),2))

assert(matricesIguales(np.diag([1,1]),np.eye(2)))
assert(matricesIguales(np.linalg.inv(np.array([[1,2],[3,4]]))@np.array([[1,2],[3,4]]),np.eye(2)))
assert(not matricesIguales(np.array([[1,2],[3,4]]).T,np.array([[1,2],[3,4]])))


In [ ]:
def rota(theta: float):
    cos = np.cos(theta)
    sen = np.sin(theta)
    
    return np.array([[cos,-sen],[sen,cos]])

In [ ]:
assert np.allclose(rota(0), np.eye(2))

assert np.allclose(
    rota(np.pi / 2),
    np.array([[0, -1],
              [1,  0]])
)

assert np.allclose(
    rota(np.pi),
    np.array([[-1,  0],
              [ 0, -1]])
)


In [ ]:
def escala(s):
    matriz = np.eye(len(s))

    for i in range(len(s)):
        matriz[i][i] = s[i]

    return matriz

In [ ]:
assert np.allclose(escala([2, 3]), np.array([[2, 0], [0, 3]]))
assert np.allclose(escala([1, 1, 1]), np.eye(3))
assert np.allclose(escala([0.5, 0.25]), np.array([[0.5, 0], [0, 0.25]]))

In [ ]:
def rota_y_escala(theta: float, s):
    return escala(s)@rota(theta)


In [ ]:
assert np.allclose(
    rota_y_escala(0, [2, 3]),
    np.array([[2, 0],
              [0, 3]])
)

assert np.allclose(
    rota_y_escala(np.pi / 2, [1, 1]),
    np.array([[0, -1],
              [1,  0]])
)

assert np.allclose(
    rota_y_escala(np.pi, [2, 2]),
    np.array([[-2,  0],
              [ 0, -2]])
)


In [ ]:
def afin(theta, s, b):
    m1 = rota_y_escala(theta, s)
    return np.array([[m1[0][0],m1[0][1], b[0]],[m1[1][0], m1[1][1], b[1]],[0,0,1]])
    

In [ ]:
assert np.allclose(
    afin(0, [1, 1], [1, 2]),
    np.array([
        [1, 0, 1],
        [0, 1, 2],
        [0, 0, 1]
    ])
)

assert np.allclose(
    afin(np.pi / 2, [1, 1], [0, 0]),
    np.array([
        [0, -1, 0],
        [1,  0, 0],
        [0,  0, 1]
    ])
)

assert np.allclose(
    afin(0, [2, 3], [1, 1]),
    np.array([
        [2, 0, 1],
        [0, 3, 1],
        [0, 0, 1]
    ])
)


In [ ]:
def trans_afin(v, theta, s, b):
    casi_res = afin(theta, s, b)@np.array([[v[0]],[v[1]],[1]])
    return np.array([casi_res[0][0], casi_res[1][0]])

In [ ]:
assert np.allclose(
    trans_afin(np.array([1, 0]), np.pi / 2, [1, 1], [0, 0]),
    np.array([0, 1])
)

assert np.allclose(
    trans_afin(np.array([1, 1]), 0, [2, 3], [0, 0]),
    np.array([2, 3])
)

assert np.allclose(
    trans_afin(np.array([1, 0]), np.pi / 2, [3, 2], [4, 5]),
    np.array([4, 7])  
)


In [ ]:
trans_afin(np.array([1, 0]), np.pi / 2, [3, 2], [4, 5])

In [ ]:
def norma (x, p):
    if p == 'inf':
        return max(map(abs ,x))
    
    res = 0 
    for xi in x:
        res += xi**p
    return res**(1/p)

In [ ]:
assert(np.allclose(norma(np.array([1,1]),2),np.sqrt(2)))
assert(np.allclose(norma(np.array([1]*10),2),np.sqrt(10)))
assert(norma(np.random.rand(10),2)<=np.sqrt(10))
assert(norma(np.random.rand(10),2)>=0)

In [ ]:
def normaliza(Xs, p):
    res = []
    for x in Xs:
        res.append(x/norma(x,p))
    return res

In [ ]:
for x in normaliza([np.array([1]*k) for k in range(1,11)],2):
    assert(np.allclose(norma(x,2),1))
for x in normaliza([np.array([1]*k) for k in range(2,11)],1):
    assert(not np.allclose(norma(x,2),1) )
for x in normaliza([np.random.rand(k) for k in range(1,11)],'inf'):
    assert( np.allclose(norma(x,'inf'),1) )

In [ ]:
def normaExacta(A, p = [1, 'inf']):
    if p == 1:
        return normaInf(A.T)
    
    elif p == 'inf':
        return normaInf(A)
    
    return None

In [ ]:
def normaMatMC(A, q, p, Np):
    m = len(A[0])
    vectors = []
    for i in range(0,Np):
        vectors.append(np.random.rand(m,1))
    
    normalizados = normaliza(vectors, p)

    multiplicados = []
    for vector in normalizados:
        multiplicados.append(calcularAx(A, vector))
    
    
    for i, vector in enumerate(multiplicados):
        multiplicados[i] = (norma(vector, q), normalizados[i])
    
    return max(multiplicados, key=lambda t: t[0])

In [340]:
nMC = normaMatMC(A=np.eye(2),q=2,p=1,Np=100000)
assert(np.allclose(nMC[0],1,atol=1e-3))
assert(np.allclose(np.abs(nMC[1][0]),1,atol=1e-3) or np.allclose(np.abs(nMC[1][1]),1,atol=1e-3))
assert(np.allclose(np.abs(nMC[1][0]),0,atol=1e-3) or np.allclose(np.abs(nMC[1][1]),0,atol=1e-3))

nMC = normaMatMC(A=np.eye(2),q=2,p='inf',Np=100000)
assert(np.allclose(nMC[0],np.sqrt(2),atol=1e-3))
assert(np.allclose(np.abs(nMC[1][0]),1,atol=1e-3) and np.allclose(np.abs(nMC[1][1]),1,atol=1e-3))

A = np.array([[1,2],[3,4]])
nMC = normaMatMC(A=A,q='inf',p='inf',Np=1000000)
assert(np.allclose(nMC[0],normaExacta(A,'inf'),rtol=2e-1)) 

In [341]:
assert(np.allclose(normaExacta(np.array([[1,-1],[-1,-1]]),1),2))
assert(np.allclose(normaExacta(np.array([[1,-2],[-3,-4]]),1),6))
assert(np.allclose(normaExacta(np.array([[1,-2],[-3,-4]]),'inf'),7))
assert(normaExacta(np.array([[1,-2],[-3,-4]]),2) is None)
assert(normaExacta(np.random.random((10,10)),1)<=10)
assert(normaExacta(np.random.random((4,4)),'inf')<=4)

In [ ]:
def condMC(A, p, Np=1000000):
    AInv = inversa(A)
    if (AInv is None):
        return None
    
    normaA = normaMatMC(A, p, p, Np)[0]
    normaAInv = normaMatMC(AInv, p, p, Np)[0]
    

    return normaA * normaAInv

In [343]:
A = np.array([[1,1],[0,1]])
A_ = np.linalg.solve(A,np.eye(A.shape[0]))
normaA = normaMatMC(A,2,2,10000)
normaA_ = normaMatMC(A_,2,2,10000)
condA = condMC(A,2,10000)
assert(np.allclose(normaA[0]*normaA_[0],condA,atol=1e-3))

A = np.array([[3,2],[4,1]])
A_ = np.linalg.solve(A,np.eye(A.shape[0]))
normaA = normaMatMC(A,2,2,10000)
normaA_ = normaMatMC(A_,2,2,10000)
condA = condMC(A,2,10000)
assert(np.allclose(normaA[0]*normaA_[0],condA,atol=1e-3))


[[1 1]
 [0 1]]
[[ 3  2]
 [ 0 -1]]


AssertionError: 

In [ ]:
def condExacta(A, p):
    AInv = inversa(A)
    normaA = normaExacta(A, p)
    normaAInv = normaExacta(AInv, p)
    

    return normaA * normaAInv

In [ ]:
A = np.random.rand(10,10)
A_ = np.linalg.solve(A,np.eye(A.shape[0]))
normaA = normaExacta(A,1)
normaA_ = normaExacta(A_,1)
condA = condExacta(A,1)
assert(np.allclose(normaA*normaA_,condA))

A = np.random.rand(10,10)
A_ = np.linalg.solve(A,np.eye(A.shape[0]))
normaA = normaExacta(A,'inf')
normaA_ = normaExacta(A_,'inf')
condA = condExacta(A,'inf')
assert(np.allclose(normaA*normaA_,condA))



In [ ]:

def sustitucionHaciaAtras(A, b):
    valoresX = np.zeros(len(b))
    for i in range(len(A)-1, -1, -1):
        cocienteActual = A[i][i]
        sumatoria = 0
        for k in range(i + 1, len(b)):
            sumatoria += A[i][k] * valoresX[k]
        valoresX[i] = (b[i] - sumatoria)/cocienteActual
    return np.array(valoresX)

def sustitucionHaciaDelante(A, b):
    valoresX = []
    for i, row in enumerate(A):
        cocienteActual = row[i]
        sumatoria = 0
        for k in range(i):
            sumatoria += A[i][k] * valoresX[k]
        valoresX.append((b[i] - sumatoria)/cocienteActual)
    return np.array(valoresX)



def res_tri(L, b, inferior=True):
    if(inferior):
        return sustitucionHaciaDelante(L,b)
    return sustitucionHaciaAtras(L,b)

In [ ]:
A = np.array([[1,0,0],[1,1,0],[1,1,1]])

b = np.array([1,1,1])
assert(np.allclose(res_tri(A,b),np.array([1,0,0])))

b = np.array([0,1,0])
assert(np.allclose(res_tri(A,b),np.array([0,1,-1])))

b = np.array([-1,1,-1])
assert(np.allclose(res_tri(A,b),np.array([-1,2,-2])))

b = np.array([-1,1,-1])
assert(np.allclose(res_tri(A,b,inferior=False),np.array([-1,1,-1])))

A = np.array([[3,2,1],[0,2,1],[0,0,1]])
b = np.array([3,2,1])
assert(np.allclose(res_tri(A,b,inferior=False),np.array([1/3,1/2,1])))

A = np.array([[1,-1,1],[0,1,-1],[0,0,1]])
b = np.array([1,0,1])
assert(np.allclose(res_tri(A,b,inferior=False),np.array([1,1,1])))



In [ ]:
def triangSup(A):
    ATriangSup = A.copy()

    for i in range(len(A)):
        for j in range(len(A[i])):
            if j < i:
                ATriangSup[i,j] = 0
    
    return ATriangSup

def triangL(A):
    L = A.copy()

    for i in range(len(A)):
        for j in range(len(A[i])):
            if j > i:
                L[i][j] = 0 
            if j == i:
                L[i][i] = 1
    
    return L

In [ ]:
def calculaLU(A):
    cant_op = 0
    m=A.shape[0]
    n=A.shape[1]
    Ac = A.copy()
    
    if m!=n:
        return None, None, 0
    
    for k in range(0, n-1):
        if A[k][k] == 0:
            return None, None, 0
        
        for i in range(k + 1, n):
            
            mi = Ac[i][k]/Ac[k][k]
            cant_op += 1
            Ac[i][k] = mi
            for j in range(k+1, m):
                Ac[i][j] = Ac[i][j] - mi * Ac[k][j]
                cant_op += 2 
    
    return triangL(Ac), triangSup(Ac), cant_op

In [ ]:
def colIdentidad(dimension, col):
    columna = np.zeros(dimension)
    columna[col] = 1
    return columna


In [ ]:
def inversa(A):
    dim = len(A)

    L,U,_ = calculaLU(A)

    if (L is None or U is None):
        return None    

    Linv = np.zeros((dim,dim))
    Uinv = np.zeros((dim,dim))

    for i in range(dim):
        colInv = res_tri(L, colIdentidad(dim, i), inferior=True)
        for j in range(dim):
            Linv[j][i] = colInv[j]

    for i in range(dim):
        if( U[i,i] == 0):
            return None

        colInv = res_tri(U, colIdentidad(dim, i), inferior=False)
        for j in range(dim):
            Uinv[j][i] = colInv[j]

    return Uinv @ Linv




In [ ]:
ntest = 10
iter = 0
while iter < ntest:
    A = np.random.random((4,4))
    A_ = inversa(A)
    if not A_ is None:

        assert(np.allclose(np.linalg.inv(A),A_))
        iter += 1

# Matriz singular devería devolver None
A = np.array([[1,2,3],[4,5,6],[7,8,9]])
assert(inversa(A) is None)